In [1]:
import csv
import serial
import numpy as np
import urllib.request 

import scipy.signal

In [ ]:
try:
    arduino = serial.Serial("/COM9",timeout=1)
    print("The port has been successfully loaded.")
except:
    print("Please check the port.")

In [ ]:
dataset = []
data_count = 0

while data_count<11:
    rawdata = []
    count = 0
    while count<100:
        rawdata.append(int(arduino.readline()))
        count+=1
    dataset.append(rawdata)
    data_count+=1

# Filtering of the input values
filtered_values = []

mean_value = np.mean(dataset[10])
value_25 = np.percentile(dataset[10], 25)
value_75 = np.percentile(dataset[10], 75)

for value in dataset[0]:
    filtered_values.append(value_75 if value > mean_value else value_25)
    
url_parameter = ""
for i in range(0,len(filtered_values)):
    if(i<len(filtered_values)-1):
        url_parameter += str(filtered_values[i])
        url_parameter += ","
    else:
        url_parameter += str(filtered_values[i])


# Extraction of the signal parameters
extracted_parameters = [np.amin(filtered_values), np.amax(filtered_values),
                        np.ptp(filtered_values), np.percentile(filtered_values, 75),
                        np.percentile(filtered_values, 25), np.median(filtered_values),
                        np.mean(filtered_values), round(np.std(filtered_values),2),
                        round(np.var(filtered_values),2), round(scipy.stats.kurtosis(filtered_values),2),
                        round(scipy.stats.skew(filtered_values),2)]

print('\nThe parameters of this signal are: ', extracted_parameters)

# Calling the web API
web_api_url = "https://pacific-harbor-19774.herokuapp.com/predict?myvar=" + str(url_parameter)
print('\nThe web API URL is: ', web_api_url)
print(str(urllib.request.urlopen(web_api_url).read()[18:22]))
print('\nThe filtered values are: ', filtered_values)

In [ ]:
arduino.close()